In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import cv2
import tensorflow as tf
import os
import shutil
from PIL import Image
import random

### Dividindo o dataset em treino e teste

In [2]:
def obter_nomes_pastas(diretorio):
    nomes_pastas = []
    for nome in os.listdir(diretorio):
        if os.path.isdir(os.path.join(diretorio, nome)):
            nomes_pastas.append(nome)
    return nomes_pastas
  
  
# Exemplo de uso
cwd = os.getcwd()


diretorio = cwd + "./data/dataset_recortado_cnn/"
pastas = obter_nomes_pastas(diretorio)

diretorio_origem = diretorio
diretorio_destino = cwd + './data/cnn_treino_categorical/'

percent = 0.8

for each in pastas:

    # Diretório contendo todas as imagens
    diretorio_imagens = diretorio_origem + f'/{each}/'

    # Diretório para salvar as imagens de treinamento
    diretorio_treinamento = diretorio_destino + f'/train/{each}/'

    # Diretório para salvar as imagens de teste
    diretorio_teste = diretorio_destino + f'/test/{each}/'

    # Lista todas as imagens no diretório
    imagens = os.listdir(diretorio_imagens)

    # Embaralha a lista de imagens
    random.shuffle(imagens)

    # Calcula o índice para dividir as imagens
    indice_divisao = int(percent * len(imagens))

    # Separa as imagens em treinamento e teste
    imagens_treinamento = imagens[:indice_divisao]
    imagens_teste = imagens[indice_divisao:]

    if not (os.path.exists(diretorio_teste)):
        os.makedirs(diretorio_teste)

    if not (os.path.exists(diretorio_treinamento)):
        os.makedirs(diretorio_treinamento)

    # Move as imagens para os diretórios correspondentes
    for imagem in imagens_treinamento:
        shutil.copy(os.path.join(diretorio_imagens, imagem),
                    diretorio_treinamento)

    for imagem in imagens_teste:
        shutil.copy(os.path.join(diretorio_imagens, imagem), diretorio_teste)



In [2]:
def segmentacaoRegiao(image):
    
    altura, largura, _ = image.shape
    
    img_tratada = cv2.GaussianBlur(image, (7,7), 7)

    x = largura//2
    y = altura//2

    
    cv_image = cv2.cvtColor(img_tratada, cv2.COLOR_BGR2GRAY)
    
              
    px_escuro, px_claro = pegarPixelClaroeEscuroRegiao(cv_image)
    
    
    # Pega o threshold com a soma do pixel mais escuro com o mais claro e divide por 2, 
    # caso seja mt baixo o valor menor que 40 ele soma os valores
    threshold = (int(px_claro) - int(px_escuro)) / 4
    

    
    region = crescimentoRegiao(cv_image, (x,y), threshold)
    

    contours, _ = cv2.findContours(region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(region)

    cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

    result_image = cv2.bitwise_and(image, image, mask=mask)
    
     
     
    return result_image


def cutNucImage(path_image, value_expand):
    nome_img_selecionada = path_image.split("/")
    nome_img_selecionada = nome_img_selecionada[(len(nome_img_selecionada)-1)]

    df = pd.read_csv(os.getcwd() + "/data/classifications.csv")
    df = df[df['image_filename'] == nome_img_selecionada]

    # Leitura da imagem com padrão RGB para preservar cores para conversão para PIL.
    img = cv2.cvtColor(cv2.imread(path_image), cv2.COLOR_BGR2RGB)

    img_cut_dict = {}

    for each in df.iterrows():
      
      posi_x = each[1]['nucleus_x']
      posi_y = each[1]['nucleus_y']
      cell_id = each[1]["cell_id"]
      
      # Normalizar o tamanho do quadrado para não pegar uma imagem completada com preto.
      x1 = max(0, posi_x - value_expand)
      y1 = max(0, posi_y - value_expand)
      x2 = min(img.shape[1], posi_x + value_expand)
      y2 = min(img.shape[0], posi_y + value_expand)
                  
      # Recortando a imagem;
      img_recortada = img[y1:y2,x1:x2]
      
      if(len(img_recortada)!= 0):
        img_cut_dict[cell_id] = img_recortada.copy()
      
      # Limpar a variavel cv2
      cv2.destroyAllWindows() 
    return img_cut_dict

def pegarPixelClaroeEscuroRegiao(imagem):
      # Encontrar o valor do pixel mais claro e mais escuro
    pixel_mais_escuro = np.min(imagem)
    pixel_mais_claro = np.max(imagem)
    
    return pixel_mais_escuro, pixel_mais_claro



def crescimentoRegiao(image, seed, threshold):
    '''
      Função para realização do crescimento por região
      
      Parâmetros:
        image (cv2): imagem para segmentar
        seed (tupla): posição central da imagem
        threshold (int): limiar de corte
      
      Return:
        image (np.array): imagem segmentada em array
    '''    
    
    
    # Pega a altura e a largura
    altura, largura = image.shape
    # Inicia matriz para pegar os px visitados
    matriz_visitados = np.zeros((altura, largura), dtype=np.uint8)
    # Matriz para pegar região segmentada
    matriz_segmentado = np.zeros_like(image)
    
    # Inicia a lista com a semente
    list_px = []
    list_px.append(seed)

    while list_px:
        # Retira um px da lista
        x, y = list_px.pop()
        # Verifica se o px foi visitado
        if not matriz_visitados[x, y]:
            # Verifica se o px esta dentro do limiar de similiadade com a semente
            # if np.linalg.norm(image[x, y] - image[seed]) < threshold:
            if np.linalg.norm(image[x, y].astype(float) - image[seed].astype(float)) < threshold:
                # Marca o px como visitado
                matriz_visitados[x, y] = 1
                # Adiciona o px na regiao segmentada
                matriz_segmentado[x, y] = image[x, y]
                # Percorre os px vizinhos do px
                for dx in range(-1, 2):
                    for dy in range(-1, 2):
                        # Verifica se o vizinho esta dentro dos limites da imagem.
                        if 0 <= x + dx < altura and 0 <= y + dy < largura:
                            # Add vizinho na lista
                            list_px.append((x + dx, y + dy))

    return matriz_segmentado
  

In [3]:
def cutDataSet(value_expand, img_name, diretorio_dataset, diretorio_dataset_recortado, path_csv):
  
  df = pd.read_csv(path_csv)
  
  df = df[df["image_filename"] == img_name]
  
  for each in df.iterrows():
  
    nome_img = each[1]['image_filename']
    nome_da_doenca = each[1]['bethesda_system']
    posi_x = each[1]['nucleus_x']
    posi_y = each[1]['nucleus_y']
    id_celular = each[1]['cell_id']  
    
    path_imagem_dataset_original = f'{diretorio_dataset}{nome_img}'

    # Onde ele vai ler cada imagem;
    img = cv2.imread(path_imagem_dataset_original)
    
    x1 = max(0, posi_x - value_expand)
    y1 = max(0, posi_y - value_expand)
    x2 = min(img.shape[1], posi_x + value_expand)
    y2 = min(img.shape[0], posi_y + value_expand)
    
    # Recortando a imagem;
    img_recortada = img[y1:y2,x1:x2]
    
    # Verifica se existe um folder no destino com o nome da doenca;
    if not os.path.exists(os.path.join(diretorio_dataset_recortado, nome_da_doenca)):
      os.mkdir(os.path.join(diretorio_dataset_recortado, nome_da_doenca))
      
    # Salva a imagem recortada no novo destino
    
    
    if(len(img_recortada)!= 0):    
      try:
        
        img_final_recortada = segmentacaoRegiao(img_recortada)
          
        cv2.imwrite(f'{diretorio_dataset_recortado}{nome_da_doenca}/' + f'{id_celular}.png', img_final_recortada)
      except:
        print(id_celular)
    # Limpar a variavel cv2
    cv2.destroyAllWindows()

In [4]:
nome_arq = os.listdir("../data/base/")


path_dataset = '../data/base/'
path_destino = "../data/segmentation_dataset_categorical/"
path_csv = '../data/classifications.csv'

for img_name in nome_arq:
  cutDataSet(value_expand=112, img_name=img_name, diretorio_dataset=path_dataset, diretorio_dataset_recortado=path_destino, path_csv=path_csv)



